In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5 import QtWidgets, uic
from PyQt5.QtGui import *
import sys
import os
from PIL import Image
import os, glob, numpy as np 
from keras.models import load_model
import socket, threading
basic_ui = uic.loadUiType("python_cnn_UI.ui")[0]

Using TensorFlow backend.


In [ ]:
#버튼 출력 윈도우창   

class WindowClass(QMainWindow, basic_ui):
    
    # 데이터셋 저장한 디렉토리
    base_dir = './KWGuideImg/dataset/train'
    categories=os.listdir(base_dir) #["기념","화도","옥의","비마","새빛","참빛","한울","연구"]
    
    def __init__(self):
        super().__init__()
        self.setupUi(self)
       
        self.cnn_result="" #이미지 분류
        self.th = QThread() 
        self.serverbtn1.clicked.connect(self.Server1)
        self.serverbtn2.clicked.connect(self.Server2)
        self.show()
    
    def binder(self,client_socket, addr): # binder함수는 서버에서 accept가 되면 생성되는 socket 인스턴스를 통해 client로 부터 데이터를 받으면 echo형태로 재송신하는 메소드

        print('Connected by', addr); # 커넥션이 되면 접속 주소가 나온다.
        try:
        # 접속 상태에서는 클라이언트로 부터 받을 데이터를 무한 대기한다.
        # 만약 접속이 끊기게 된다면 except가 발생해서 접속이 끊기게 된다.
            #while True:
            data = client_socket.recv(4)  # socket의 recv함수는 연결된 소켓으로부터 데이터를 받을 대기하는 함수

            length = int.from_bytes(data, "big")    # 최초 4바이트는 전송할 데이터의 크기이다. 그 크기는 big 엔디언으로 byte에서 int형식으로 변환한다.C#의 BitConverter는 big엔디언으로 처리된다.

            data = client_socket.recv(length) # 다시 데이터 수신

            msg = data.decode() # 수신된 데이터를 str형식으로 decode한다.

            print('Received from', addr, msg) # 수신된 메시지를 콘솔에 출력한다.

            global MSG 
            MSG = msg
            print("MSG : ",MSG)

            data = msg.encode() # 바이너리(byte)형식으로 변환한다.

            length = len(data)      # 바이너리의 데이터 사이즈를 구한다.

            client_socket.sendall(length.to_bytes(4, byteorder='big'))  # 데이터 사이즈를 big 엔디언 형식으로 byte로 변환한 다음 전송한다.(※이게 버그인지 big을 써도 little엔디언으로 전송된다.)

            client_socket.sendall(data) # 데이터를 클라이언트로 전송한다. cnn 결과값
            #client_socket.close() 
        except:

            print("except : " , addr)  # 접속이 끊기면 except가 발생한다.
                
        finally:

            client_socket.close() # 접속이 끊기면 socket 리소스를 닫는다.
            
    def Server1(self):
        
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) # 소켓을 만든다.

        server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) # 소켓 레벨과 데이터 형태를 설정한다.

        server_socket.bind(('', 9999)); # 서버는 복수 ip를 사용하는 pc의 경우는 ip를 지정하고 그렇지 않으면 None이 아닌 ''로 설정한다. 포트는 pc내에서 비어있는 포트를 사용한다. cmd에서 netstat -an | find "LISTEN"으로 확인할 수 있다.

        server_socket.listen() # server 설정이 완료되면 listen를 시작한다.
        
        print("client 접속중...")
        
        
        try:

            #while True: # 서버는 여러 클라이언트를 상대하기 때문에 무한 루프를 사용한다.

            client_socket, addr = server_socket.accept() # client로 접속이 발생하면 accept가 발생한다. client 소켓과 addr(주소)를 튜플로 받는다
            self.th = threading.Thread(target=self.binder, args = (client_socket,addr))  # 쓰레드를 이용해서 client 접속 대기를 만들고 다시 accept로 넘어가서 다른 client를 대기한다.
            print("스레드 시작")
            self.th.start()
            
        except:
            print("server1")
            server_socket.close() # 에러가 발생하면 서버 소켓을 닫는다
        finally:

            server_socket.close() # 에러가 발생하면 서버 소켓을 닫는다.
    
    def binder2(self,client_socket, addr): # binder함수는 서버에서 accept가 되면 생성되는 socket 인스턴스를 통해 client로 부터 데이터를 받으면 echo형태로 재송신하는 메소드
    
        print('Connected by', addr); # 커넥션이 되면 접속 주소가 나온다.
        try:
        # 접속 상태에서는 클라이언트로 부터 받을 데이터를 무한 대기한다.
        # 만약 접속이 끊기게 된다면 except가 발생해서 접속이 끊기게 된다.
            #while True:
            data = client_socket.recv(4)  # socket의 recv함수는 연결된 소켓으로부터 데이터를 받을 대기하는 함수

            length = int.from_bytes(data, "big")    # 최초 4바이트는 전송할 데이터의 크기이다. 그 크기는 big 엔디언으로 byte에서 int형식으로 변환한다.C#의 BitConverter는 big엔디언으로 처리된다.

            data = client_socket.recv(length) # 다시 데이터 수신

            msg = data.decode() # 수신된 데이터를 str형식으로 decode한다.

            print('Received from', addr, msg) # 수신된 메시지를 콘솔에 출력한다

            #훈련된 모델을 사용하여 이미지 예측
            msg=self.cnn_result

            data = msg.encode() # 바이너리(byte)형식으로 변환한다.

            length = len(data)      # 바이너리의 데이터 사이즈를 구한다.

            client_socket.sendall(length.to_bytes(4, byteorder='big'))  # 데이터 사이즈를 big 엔디언 형식으로 byte로 변환한 다음 전송한다.(※이게 버그인지 big을 써도 little엔디언으로 전송된다.)

            client_socket.sendall(data) # 데이터를 클라이언트로 전송한다. cnn 결과값
        except:

            print("except : " , addr)  # 접속이 끊기면 except가 발생한다.

        finally:

            client_socket.close() # 접속이 끊기면 socket 리소스를 닫는다.
        
    def Server2(self):
        
        print(MSG)
        img = Image.open(MSG)
        img = img.convert("RGB")
        image_w = 64
        image_h = 64 #크기지정
        img = img.resize((image_w, image_h)) #사이즈 재조정
        data = np.asarray(img)
        X=np.array(data)

        X = X.astype("float") / 256
        X = X.reshape(-1, 64, 64,3)

        #훈련된 모델을 사용하여 이미지 예측
        model=load_model('Building_cnn.h5')
        predictions = model.predict(X)  #X이미지 레이블 예측
        predictions[0]
        result = np.argmax(predictions[0])    # 예측 값중 가장 높은 신뢰도를 가진 레이블
        base_dir = './KWGuideImg/dataset/train'
        categories=os.listdir(base_dir) #["기념","화도","옥의","비마","새빛","참빛","한울","연구"]
        self.cnn_result = str(categories[result])       # 수신된 메시지 앞에 「echo:」 라는 메시지를 붙힌다.
        print(self.cnn_result)

        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) # 소켓을 만든다.

        server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) # 소켓 레벨과 데이터 형태를 설정한다.

        server_socket.bind(('', 9999)); # 서버는 복수 ip를 사용하는 pc의 경우는 ip를 지정하고 그렇지 않으면 None이 아닌 ''로 설정한다. 포트는 pc내에서 비어있는 포트를 사용한다. cmd에서 netstat -an | find "LISTEN"으로 확인할 수 있다.

        server_socket.listen() # server 설정이 완료되면 listen를 시작한다.

        print("client 접속중...")
        
        try:

            while True: # 서버는 여러 클라이언트를 상대하기 때문에 무한 루프를 사용한다.

                client_socket, addr = server_socket.accept() # client로 접속이 발생하면 accept가 발생한다. client 소켓과 addr(주소)를 튜플로 받는다
                th = threading.Thread(target=self.binder2, args = (client_socket,addr))  # 쓰레드를 이용해서 client 접속 대기를 만들고 다시 accept로 넘어가서 다른 client를 대기한다.

                th.start()
        except:
            print("server2")
            server_socket.close() # 에러가 발생하면 서버 소켓을 닫는다
        finally:
            server_socket.close() # 에러가 발생하면 서버 소켓을 닫는다.
                           
app = QApplication([])
ex = WindowClass()
sys.exit(app.exec_())


client 접속중...
스레드 시작
Connected by ('127.0.0.1', 54478)
Received from ('127.0.0.1', 54478) C:\github\KW_GuideBook\KWGuideImg\dataset\test\test_img2.jpg
MSG :  C:\github\KW_GuideBook\KWGuideImg\dataset\test\test_img2.jpg
C:\github\KW_GuideBook\KWGuideImg\dataset\test\test_img2.jpg
saebit
client 접속중...
Connected by ('127.0.0.1', 54482)
Received from ('127.0.0.1', 54482) hello
